In [6]:
# !pip install qwen_vl_utils
# pip install -U git+https://github.com/huggingface/transformers
# cuda 12.2 
# conda install pytorch torchvision torchaudio pytorch-cuda=12.1 -c pytorch -c nvidia
# transformers==4.46.2
# GPU Memory Usage 7278MiB (test image)
# pdf image resized 1/5times 4838MiB
# pdf image resized 1/3times 5274MiB
# pdf image resized 1/2times 6554MiB
# 1/3times image; 2 img input 6686MiB

In [12]:
import torch
torch.cuda.is_available()

import glob
import os
import pandas as pd

import datetime

In [2]:
# pip install byaldi accelerate flash-attn qwen_vl_utils pdf2image
from qwen_vl_utils import process_vision_info
# from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor
## if RuntimeError, https://stackoverflow.com/a/65277647 (just uninstall numpy untill none then install again)

/home/ecmdev/anaconda3/envs/rag_sim/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
model = Qwen2VLForConditionalGeneration.from_pretrained(
    "Qwen2-VL-2B-Instruct", torch_dtype="auto", device_map="auto"
)
#load = GPU Memory Usage 4638MiB

`Qwen2VLRotaryEmbedding` can now be fully parameterized by passing the model config through the `config` argument. All other arguments will be removed in v4.46
Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  3.30it/s]


In [4]:
# default processer
processor = AutoProcessor.from_pretrained("Qwen2-VL-2B-Instruct")


#### Process for individual query

In [43]:
text_query = "When molten iron temperature is 1400°C, what should the CaO, SiO2 ratio be for optimal output?"
image_input = glob.glob(f'../ColPali/search_results/resized/{text_query}_results_0_*_resized.jpg')[0]

text_query_1 = "When molten iron temperature is 1400°C, what should the CaO, SiO2 ratio be for optimal output?"
image_input_1 = glob.glob(f'../ColPali/search_results/resized/{text_query_1}_results_1_*_resized.jpg')[0]

image_input, image_input_1

IndexError: list index out of range

In [15]:
# text_query = "When molten iron temperature is 1400°C, what should the C\S ratio be for optimal output?"

In [16]:
messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "image",
                "image": image_input,
            },
            {
                "type": "image",
                "image": image_input_1,
            },
            {"type": "text", 
             "text": text_query},
        ],
    }
]

In [17]:
text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)

In [19]:
image_inputs, video_inputs = process_vision_info(messages)

In [20]:
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)
inputs = inputs.to("cuda")

In [21]:
image_inputs

[<PIL.Image.Image image mode=RGB size=560x784>,
 <PIL.Image.Image image mode=RGB size=560x784>]

In [22]:
# Inference: Generation of the output
generated_ids = model.generate(**inputs, max_new_tokens=128)

generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)
print(output_text)

['According to the table in the text, the optimal CaO-SiO2 ratio for optimal output at a molten iron temperature of 1400°C is 1.58.']


In [23]:
print(output_text[0])

According to the table in the text, the optimal CaO-SiO2 ratio for optimal output at a molten iron temperature of 1400°C is 1.58.


#### Work on list of queries and img input

In [6]:
query_file = '../ColPali/data/250113_query_list.txt'
query_file_name = query_file.split('/')[-1].split('.')[0]

## open query list file
with open(query_file, 'r') as f:
    query_list = f.readlines()
f.close()

In [7]:
date = datetime.datetime.now().strftime('%y%m%d')

In [8]:
## create new txt file for results
results_file = f'results/{date}_RAG_results.txt'

if os.path.exists(results_file):
    print('file already exist')
else:
    with open(results_file, 'w') as f:
        f.write('query\tgenerated answer\timg input list\n')
    f.close()
    print(f'{results_file} created :)')

file already exist


In [12]:
for idx, query in enumerate(query_list[1:]):
    query = query.strip()
    print(f'Q{idx+1}: ', query)

    # get the image input data dir
    image_input_0 = glob.glob(f'../ColPali/search_results/resized/{query_file_name}_{idx}_0_*_resized.jpg')[0]
    image_input_1 = glob.glob(f'../ColPali/search_results/resized/{query_file_name}_{idx}_1_*_resized.jpg')[0]

    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "image",
                    "image": image_input_0,
                },
                {
                    "type": "image",
                    "image": image_input_1,
                },
                {"type": "text", 
                "text": query},
            ],
        }
    ]

    text = processor.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )

    image_inputs, video_inputs = process_vision_info(messages)

    inputs = processor(
        text=[text],
        images=image_inputs,
        videos=video_inputs,
        padding=True,
        return_tensors="pt",
    )
    inputs = inputs.to("cuda")

    # Inference: Generation of the output
    generated_ids = model.generate(**inputs, max_new_tokens=128)

    generated_ids_trimmed = [
        out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
    ]
    output_text = processor.batch_decode(
        generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
    )

    # record onto the existing file
    print('\tA: ', output_text[0])
    with open(results_file, 'a') as f:
        f.write(f'{query}\t{output_text}\t{[image_input_0, image_input_1]}\n')
    f.close()

    print('====RECORD UPDATED====')

Q1:  When molten iron temperature is 1400℃, what should the C-S ratio be for optimal output?
	A:  According to the text, the C-S ratio should be 0.7 for optimal output when molten iron temperature is 1400°C.
====RECORD UPDATED====
Q2:  When molten iron temperature is 1500℃, how much of Al2O3 content for the optimal slag viscosity?
	A:  According to the text, the optimal slag viscosity occurs when the slag contains 25-30 percent alumina, with high viscosity and liquidus temperatures. Therefore, the optimal slag viscosity can be achieved by adding 25-30 percent of alumina to the slag.
====RECORD UPDATED====
Q3:  How much does the reducing agent ratio decrease when the average Top Gas temperature increases by 20°C?
	A:  According to Trense and Rosborough, the top gas CO/CO2 ratio increases by 1 percent for a 40°C decrease in the flame temperature below the base case and decreases by 5 percent for each 40°C increase above the base case.
====RECORD UPDATED====
Q4:  What is the ideal coke pa

### 250124 work on colpali result txt file

In [7]:
folder_list = glob.glob("**/", root_dir="../ColPali/search_results/")
folder_list

['250123_simulation_test/', 'resized/', '250113_query_list/']

In [10]:
file_name = folder_list[0][:-1] ###############3 change the folderlist idx
result_file = f'../ColPali/search_results/{file_name}/{file_name}_results.txt'
result_file

'../ColPali/search_results/250123_simulation_test/250123_simulation_test_results.txt'

In [11]:
## bring the colpali result
data = pd.read_csv(result_file, sep='\t')
data.head()

,query_no,query,doc_id,doc_name,search_rank,searched_page,score
0,0,"When molten iron temperature is 1400℃, what sh...",1,ref_documents/Principles of Blast Furnace Iron...,0,455,21.375
1,0,"When molten iron temperature is 1400℃, what sh...",1,ref_documents/Principles of Blast Furnace Iron...,1,374,21.125
2,0,"When molten iron temperature is 1400℃, what sh...",2,ref_documents/Beyond the blast furnace 1992.pdf,2,221,21.125
3,1,"When molten iron temperature is 1500℃, how muc...",2,ref_documents/Beyond the blast furnace 1992.pdf,0,156,25.375
4,1,"When molten iron temperature is 1500℃, how muc...",1,ref_documents/Principles of Blast Furnace Iron...,1,67,25.250


In [16]:
query_list = data['query'].unique()
query_list

array(['When molten iron temperature is 1400℃, what should the C-S ratio be for optimal output?',
       'When molten iron temperature is 1500℃, how much of Al2O3 content for the optimal slag viscosity?',
       'How much does the reducing agent ratio decrease when the average Top Gas temperature increases by 20°C?',
       'What is the ideal coke particle size for improving the fluidity of the melting matter at the hearth of the blast furnace?',
       'What actions can be taken to lower the Si(%)?',
       'What impact does the accretion have on the inside of the furnace?',
       'What operational actions should be taken to reduce the heat load on the blast furnace body?',
       'What operational effects occur when the Coke Wf-Di is below the standard level?',
       'When 5 tons of water spraying is applied to the furnace, how many tons of pulverized coal should be injected to compensate for the heat loss?',
       'What actions can be taken to prevent the bottom sidewall-temperat

In [13]:
date = datetime.datetime.now().strftime('%y%m%d')
date

'250124'

In [14]:
## create new txt file for results
results_file = f'results/{file_name}_{date}_RAG_results.txt'

if os.path.exists(results_file):
    print('file already exist')
else:
    with open(results_file, 'w') as f:
        f.write('query\tgenerated answer\timg input list\n')
    f.close()
    print(f'{results_file} created :)')

results/250123_simulation_test_250124_RAG_results.txt created :)


In [18]:
glob.glob(f'../ColPali/search_results/{file_name}/{file_name}_{0}_*.jpg')

['../ColPali/search_results/250123_simulation_test/250123_simulation_test_0_2_221.jpg',
 '../ColPali/search_results/250123_simulation_test/250123_simulation_test_0_1_374.jpg',
 '../ColPali/search_results/250123_simulation_test/250123_simulation_test_0_0_455.jpg']

In [20]:
for idx, query in enumerate(query_list):
    query = query.strip()
    print(f'Q{idx+1}: ', query)

    # get the image input data dir
    image_input_list = glob.glob(f'../ColPali/search_results/{file_name}/{file_name}_{idx}_*.jpg')

    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "image",
                    "image": image_input_list[0],
                },
                {
                    "type": "image",
                    "image": image_input_list[1],
                },
                {
                    "type": "image",
                    "image": image_input_list[2],
                },
                {"type": "text", 
                "text": query},
            ],
        }
    ]

    text = processor.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )

    image_inputs, video_inputs = process_vision_info(messages)

    inputs = processor(
        text=[text],
        images=image_inputs,
        videos=video_inputs,
        padding=True,
        return_tensors="pt",
    )
    inputs = inputs.to("cuda")

    # Inference: Generation of the output
    generated_ids = model.generate(**inputs, max_new_tokens=128)

    generated_ids_trimmed = [
        out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
    ]
    output_text = processor.batch_decode(
        generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
    )

    # record onto the existing file
    print('\tA: ', output_text[0])
    with open(results_file, 'a') as f:
        f.write(f'{query}\t{output_text}\t{image_input_list}\n')
    f.close()

    print('====RECORD UPDATED====')

Q1:  When molten iron temperature is 1400℃, what should the C-S ratio be for optimal output?
	A:  The optimal C-S ratio for optimal output when molten iron temperature is 1400°C is 0.06%.
====RECORD UPDATED====
Q2:  When molten iron temperature is 1500℃, how much of Al2O3 content for the optimal slag viscosity?
	A:  According to the figure in the text, the optimal slag viscosity is achieved when the Al2O3 content is around 35.6%.
====RECORD UPDATED====
Q3:  How much does the reducing agent ratio decrease when the average Top Gas temperature increases by 20°C?
	A:  The reducing agent ratio decreases by 1.5% when the average Top Gas temperature increases by 20°C.
====RECORD UPDATED====
Q4:  What is the ideal coke particle size for improving the fluidity of the melting matter at the hearth of the blast furnace?
	A:  The ideal coke particle size for improving the fluidity of the melting matter at the hearth of the blast furnace is 40 mm.
====RECORD UPDATED====
Q5:  What actions can be take

#### Work on one query with several modifications

In [9]:
query_file = '../ColPali/data/250113_query_list_0_prompt.txt'

## open query list file
with open(query_file, 'r') as f:
    query_list = f.readlines()
f.close()

In [10]:
## create new txt file for results
query_file_name = query_file.split('/')[-1].split('.')[0][:-7]
results_file = f'results/{query_file_name}_RAG_results.txt'

if os.path.exists(results_file):
    print('file already exist')
else:
    with open(results_file, 'w') as f:
        f.write('query\tgenerated answer\n')
    f.close()
    print(f'{results_file} created :)')

results/250113_query_list_0_RAG_results.txt created :)


In [11]:
# get the image input data dir
image_input_0 = glob.glob(f'../ColPali/search_results/resized/{query_file_name}_0_*_resized.jpg')[0]
image_input_1 = glob.glob(f'../ColPali/search_results/resized/{query_file_name}_1_*_resized.jpg')[0]
image_input_2 = glob.glob(f'../ColPali/search_results/resized/{query_file_name}_2_*_resized.jpg')[0]

with open(results_file, 'a') as f:
    f.write(f'{image_input_0}, {image_input_1}, {image_input_2}\n')
f.close()

for idx, query in enumerate(query_list[1:]):
    query = query.strip()
    print(f'Q{idx+1}: ', query)

    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "image",
                    "image": image_input_0,
                },
                {
                    "type": "image",
                    "image": image_input_1,
                },
                {
                    "type": "image",
                    "image": image_input_2,
                },
                {"type": "text", 
                "text": query},
            ],
        }
    ]

    text = processor.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )

    image_inputs, video_inputs = process_vision_info(messages)

    inputs = processor(
        text=[text],
        images=image_inputs,
        videos=video_inputs,
        padding=True,
        return_tensors="pt",
    )
    inputs = inputs.to("cuda")

    # Inference: Generation of the output
    generated_ids = model.generate(**inputs, max_new_tokens=256)

    generated_ids_trimmed = [
        out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
    ]
    output_text = processor.batch_decode(
        generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
    )

    # record onto the existing file
    print('\tA: ', output_text[0])
    with open(results_file, 'a') as f:
        f.write(f'{query}\t{output_text}\n')
    f.close()

    print('====RECORD UPDATED====')

Q1:  When molten iron temperature is 1400℃, what should the CaO-SiO2 ratio be for optimal output?
	A:  According to the text, the optimal CaO-SiO2 ratio for optimal output is 0.8.
====RECORD UPDATED====
Q2:  What is the exact ratio of CaO-SiO2 for maximum output at 1400 degrees of molten iron temperature?
	A:  The exact ratio of CaO-SiO2 for maximum output at 1400 degrees of molten iron temperature is 1.5.
====RECORD UPDATED====
Q3:  For molten iron at 1400℃, what is the ideal CaO-SiO2 ratio to achieve optimal output?
	A:  According to the text, the ideal CaO-SiO2 ratio to achieve optimal output is 0.8.
====RECORD UPDATED====
Q4:  What CaO-SiO2 ratio should be used when the molten iron temperature is 1400℃ for maximum efficiency?
	A:  According to the text, the CaO-SiO2 ratio should be 0.8 for maximum efficiency when the molten iron temperature is 1400℃.
====RECORD UPDATED====
Q5:  At a molten iron temperature of 1400℃, what CaO-SiO2 ratio ensures the best output?
	A:  According to the